In [316]:
import pandas as pd # модуль для роботи із таблицями
import nltk 
import numpy as np # математичний і статистичний модуль
import re # оброблення регулярних виразів
from nltk.stem import wordnet # для здійснення Лемматизації (приведення словоформи до її первісної словникової форми)
from sklearn.feature_extraction.text import CountVectorizer # викликання bow-методу (Bag of Words - вилучення об’єктів із текстових документів)
from sklearn.feature_extraction.text import TfidfVectorizer # викликання tfidf (оцінка важливості слова в контексті документа)
from nltk import pos_tag # визначення частини мови
from sklearn.metrics import pairwise_distances # обчислює відстані між відповідними елементами двох масивів.
from nltk import word_tokenize # ділить рядок на підрядки шляхом розділення на вказаний рядок
from nltk.corpus import stopwords # пошук стоп-слів

In [317]:
df=pd.read_excel('dialog_talk_agent.xlsx') # проводим зчитування тестової бази даних (ТБД)
df[1:101]

,Context,Text Response
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,NaN
3,Describe yourself,NaN
4,tell me about yourself,NaN
5,all about you,NaN
...,...,...
96,be more clever,NaN
97,study,NaN
98,NaN,NaN
99,you're attractive,Thank you! What a sweet thing to say.


In [318]:
df.shape[0], df.shape[1] # вказуєм на розмір ТБД (стовпці, колонки)

(1592, 2)

In [319]:
df.ffill(axis = 0,inplace=True) # заповнюємо пусті ячейки попередніми значеннями (відтворення задуманого змісту таблиці)
df

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder
...,...,...
1587,can we chat,Talking is what I do best.
1588,I'll be back in a few minutes,I'll be waiting.
1589,I'll be back,All right. I'll be here.
1590,I'll get back to you in a moment,Till next time.


In [320]:
df1=df.head(10) # копіюємо перші 10 рядків ТБД

In [321]:
# функція приведення тексту до нижнього регістру із подальшим видаленням спеціальних символів (залишаємо тільки текст)

def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [322]:
step1(df1['Context']) # опрацьовуємо першу колонку ТБД через функцію, подану вище

tell me about your personality
i want to know you better
define yourself
describe yourself
tell me about yourself
all about you
tell me some stuff about you
talk some stuff about you
talk about yourself
about yourself


In [323]:
# токенізація слів 
s='tell me about your personality'
words=word_tokenize(s)
words

['tell', 'me', 'about', 'your', 'personality']

In [324]:
lemma = wordnet.WordNetLemmatizer() # виклик лемматизації
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [325]:
pos_tag(nltk.word_tokenize(s),tagset = None) # знаходимо для токенізованих слів їх частини мови

[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [326]:
# функція нормалізації тексту (приведення до виду, зручного для програмного опрацювання)
def text_normalization(text):
    text=str(text).lower() # нижній регістр
    spl_char_text=re.sub(r'[^ a-z]','',text) # видалення спецсимволів
    tokens=nltk.word_tokenize(spl_char_text) # токенізація
    lema=wordnet.WordNetLemmatizer() # лемматизація
    tags_list=pos_tag(tokens,tagset=None) # частини мови
    lema_words=[]   # створюєм пустий список
    for token,pos_token in tags_list: 
        if pos_token.startswith('V'):  # Дієслово
            pos_val='v'
        elif pos_token.startswith('J'): # Прикметник
            pos_val='a'
        elif pos_token.startswith('R'): # Прислівник
            pos_val='r'
        else:
            pos_val='n' # Іменник
            
        lema_token=lema.lemmatize(token,pos_val) # лемматизація
        lema_words.append(lema_token) # доповнення лемматизованих слів  у lema_words
    
    return " ".join(lema_words) # отримання повних речень із токенів (складаєм все назад)

In [327]:
text_normalization('telling you some stuff about me') # тестовий виклик функції нормалізації тексту

'tell you some stuff about me'

In [328]:
df['lemmatized_text']=df['Context'].apply(text_normalization) # створення нового стопця із лемматизованих речень
df.head(5) 

,Context,Text Response,lemmatized_text
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you good
2,Define yourself,I can help you work smarter instead of harder,define yourself
3,Describe yourself,I can help you work smarter instead of harder,describe yourself
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself


In [329]:
# all the stop words we have 
# вивід усіх стоп-слів (загальновживаних слів, що ігнорує система при індексуванні записів для пошуку)

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Bag of words (bow)

Метод вилучення ознак за допомогою текстових даних (відстежуємо кількість слів і нехтуємо граматичними деталями та порядком слів)


In [330]:
cv = CountVectorizer() # виклик count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [331]:
# виведення усіх ункальних слів із ТБД

features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [332]:
Question ='Will you help me and tell me about yourself more' # тестовий запит

In [333]:
# переіврка присутності тестових слів

Q=[]
a=Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q)

In [334]:
Question_lemma = text_normalization(b) # нормалізація тестового запиту
Question_bow = cv.transform([Question_lemma]).toarray() # виклик Bag of words

In [335]:
Question_bow # "мішок слів для тестового запиту" - матриця 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Подібність (cosine similarity)

In [336]:
# косинусна близькість для тестового запиту

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.25819889],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [337]:
df['similarity_bow']=cosine_value # додаєм нову колонку для отриманих косинусних відстаней

In [338]:
df_simi = pd.DataFrame(df, columns=['Text Response','similarity_bow']) # об'єднуємо ТБД із колонкою, поданою вище
df_simi

,Text Response,similarity_bow
0,Just think of me as the ace up your sleeve.,0.258199
1,I can help you work smarter instead of harder,0.000000
2,I can help you work smarter instead of harder,0.000000
3,I can help you work smarter instead of harder,0.000000
4,I can help you work smarter instead of harder,0.288675
...,...,...
1587,Talking is what I do best.,0.000000
1588,I'll be waiting.,0.000000
1589,All right. I'll be here.,0.000000
1590,Till next time.,0.000000


In [339]:
df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False) # сортуєм значення колонки (у порядку спадання)
df_simi_sort.head(5)

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248


In [340]:
threshold = 0.2 # підбираємо значення збіжності і виводим найбільш відповідні рядки
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold[:5] # вивід п'яти перших рядків для сортованої і доповненохї ТБД

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248


In [341]:
index_value = cosine_value.argmax() # індекс найбільного значення
index_value

194

In [342]:
Question # тестове запитання

'Will you help me and tell me about yourself more'

In [343]:
df['Text Response'].loc[index_value] # відповідь, взята із ТБД за індексом найбільної схожості

"I'm glad to help. What can I do for you?"

# Tf-idf (term frequency- inverse document frequency)
Оцінка важливості слова в контексті документа ( якщо слово зустрічається в будь-якому документі часто, при цьому зустрічаючись рідко у всіх інших документах - це слово має велику значимість для того самого документа).

In [344]:
Question1 ='Tell me about yourself.' # Тестове запитання №2

In [345]:
Question_lemma1 = text_normalization(Question1) # нормалізація
Question_tfidf = tfidf.transform([Question_lemma1]).toarray() # застосування tf-idf

In [346]:
tfidf=TfidfVectorizer() # запуск tf-id для векторизації тексту
x_tfidf=tfidf.fit_transform(df['lemmatized_text']).toarray() # перевід даних у масив

In [347]:
# отримуємо усі унікальні слова із їх рахунком 

df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
df_tfidf.head()

,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0.0,0.407572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.330555,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.218768,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.641790,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.641790,0.0
4,0.0,0.453790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.608937,0.0


# Подібність (cosine similarity)
міра подібності між двома векторами.


In [348]:
cos=1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')
cos

array([[0.56511191],
       [0.        ],
       [0.39080996],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [349]:
df['similarity_tfidf']=cos # вивід нової колонки косинусної близькості для обланого тестового запитання
df_simi_tfidf = pd.DataFrame(df, columns=['Text Response','similarity_tfidf']) 
df_simi_tfidf

,Text Response,similarity_tfidf
0,Just think of me as the ace up your sleeve.,0.565112
1,I can help you work smarter instead of harder,0.000000
2,I can help you work smarter instead of harder,0.390810
3,I can help you work smarter instead of harder,0.390810
4,I can help you work smarter instead of harder,1.000000
...,...,...
1587,Talking is what I do best.,0.000000
1588,I'll be waiting.,0.000000
1589,All right. I'll be here.,0.000000
1590,Till next time.,0.000000


In [350]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False) # сортування
df_simi_tfidf_sort.head(10)

,Text Response,similarity_tfidf
4,I can help you work smarter instead of harder,1.000000
16,I can help you work smarter instead of harder,0.771758
9,I can help you work smarter instead of harder,0.759428
8,I can help you work smarter instead of harder,0.651909
379,I should get one. It's all work and no play la...,0.594479
500,The virtual world is my playground. I'm always...,0.590474
0,Just think of me as the ace up your sleeve.,0.565112
6,I can help you work smarter instead of harder,0.514553
48,I'm not programmed for that exact question. Tr...,0.445403
24,"I'm a relatively new bot, but I'm wise beyond ...",0.434832


In [351]:
threshold = 0.2 # фільтрування за threshold
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,Text Response,similarity_tfidf
4,I can help you work smarter instead of harder,1.000000
16,I can help you work smarter instead of harder,0.771758
9,I can help you work smarter instead of harder,0.759428
8,I can help you work smarter instead of harder,0.651909
379,I should get one. It's all work and no play la...,0.594479
500,The virtual world is my playground. I'm always...,0.590474
0,Just think of me as the ace up your sleeve.,0.565112
6,I can help you work smarter instead of harder,0.514553
48,I'm not programmed for that exact question. Tr...,0.445403
24,"I'm a relatively new bot, but I'm wise beyond ...",0.434832


In [352]:
index_value1 = cos.argmax() # знаходження індекса значення із найбільною подібністю 
index_value1

4

In [353]:
Question1 # тестове запитання 

'Tell me about yourself.'

In [354]:
df['Text Response'].loc[index_value1]  # відповідь за оптимальним індексом видозміненої ТБД

'I can help you work smarter instead of harder'

# Програмна модель Bag of Words 

In [355]:
# Фунція нормалізації тексту (введеного запитання)

def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_word.append(lema_token)
    return " ".join(lema_word)

In [356]:
# Функція виводу значень оптимальних індексів (навчена модель)

def chat_bow(text):
    s=stopword_(text)
    lemma=text_normalization(s) # нормалізація
    bow=cv.transform([lemma]).toarray() # застосування bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value=cosine_value.argmax() # отрмання індекса 
    return df['Text Response'].loc[index_value]

In [357]:
chat_bow('hi there') # програмний запит до чат-боту із подальшим виводом оптимальної відповіді

'Hey!'

In [358]:
chat_bow('hope to see u soon')

'Bye.'

# Програмна модель tf-idf

In [359]:
# функція опрацювання програмних запитів

def chat_tfidf(text):
    lemma=text_normalization(text)
    tf=tfidf.transform([lemma]).toarray()
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine')
    index_value=cos.argmax()
    return df['Text Response'].loc[index_value]

In [360]:
chat_tfidf('hi')

'Hey!'

In [361]:
chat_tfidf('can you help me?')

"I'm glad to help. What can I do for you?"

In [362]:
chat_tfidf('thanks for your support!')

"It's my pleasure to help."